In [1]:
GLOVE = '/content/gdrive/MyDrive/MC934/glove/glove.6B.300d.txt'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!git clone https://0ad3bca4dadfa2ed70b15ac7d82e34f0e17b0169@github.com/fake-news-deep-learning/fake-news-classifier.git

Cloning into 'fake-news-classifier'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 99 (delta 47), reused 86 (delta 37), pack-reused 0
Unpacking objects: 100% (99/99), done.
Checking out files: 100% (16/16), done.


In [8]:
%cd /content/fake-news-classifier
!git checkout train
%cd src/

/content/fake-news-classifier
Branch 'train' set up to track remote branch 'train' from 'origin'.
Switched to a new branch 'train'
/content/fake-news-classifier/src


In [10]:
import train

In [11]:
model, history = train.train_driver(GLOVE)

100%|██████████| 828/828 [00:00<00:00, 1132708.32it/s]

Fitting Tokenizer to dataset.


removing to from vocab
removing and from vocab
removing of from vocab
removing a from vocab
removing that from vocab
removing in from vocab
removing is from vocab
removing i from vocab
removing we from vocab
removing for from vocab
removing you from vocab
removing it from vocab
removing have from vocab
removing this from vocab
removing on from vocab
removing are from vocab
removing be from vocab
removing with from vocab
removing not from vocab
removing as from vocab
removing but from vocab
removing was from vocab
removing they from vocab
removing our from vocab
removing what from vocab
removing he from vocab
removing about from vocab
removing at from vocab
removing do from vocab
removing so from vocab
removing people from vocab
removing will from vocab
removing by from vocab
removing going from vocab
removing from from vocab
removing has from vocab
removing who from vocab
removing all from vocab
removing think from vocab
removing or from vocab
removing if from vocab
removing president 

400000it [00:34, 11628.97it/s]
100%|██████████| 6699/6699 [00:00<00:00, 417461.18it/s]


word2vec embeds have len 300
Translating vocabulary
Could not find 226 words in GLOVE.
Converting text from train set to sequences.


100%|██████████| 98/98 [00:00<00:00, 4462.32it/s]


Converting text from valid set to sequences.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_kernel_1_layer (Conv2 (None, 70, 300, 36)       10836     
_________________________________________________________________
conv2d_kernel_2_layer (Conv2 (None, 70, 300, 36)       777636    
_________________________________________________________________
conv2d_kernel_3_layer (Conv2 (None, 70, 300, 36)       1166436   
_________________________________________________________________
conv2d_kernel_5_layer (Conv2 (None, 70, 300, 36)       1944036   
_________________________________________________________________
dropout_layer (Dropout)      (None, 70, 300, 36)       0         
_________________________________________________________________
flatten (Flatten)            (None, 756000)            0         
_________________________________________________________________
linear_laye

In [15]:
import json

In [22]:
history_dict = {}

for cat, items in history.history.items():
    items_ = [str(item) for item in items]
    history_dict[cat] = items_

In [29]:
with open('/content/baseline_history.json', 'w') as out_file:
  json.dump(history_dict, out_file, indent=4)

In [27]:
model.save('/content/')

INFO:tensorflow:Assets written to: /content/assets


# Avaliação do modelo no conjunto de teste

In [39]:
import numpy as np
from tqdm import tqdm

from word2vec import prepare_tokenizer, text_to_sequence
from utils import load_glove

In [37]:
def eval(model):
    tokenizer = prepare_tokenizer()
    word2seq = load_glove(tokenizer.word_index, GLOVE)

    print(f'Converting text from test set to sequences.')
    with open(f'../data/processed/test.json') as input_file:
        dataset = json.load(input_file)

    test_x = []
    test_y = []

    for entry in tqdm(dataset.values()):

        sequence = text_to_sequence(
            entry['text'],
            tokenizer.word_index,
            word2seq,
        )

        test_x.append(sequence)
        test_y.append(0 if entry['label'] == 'fake' else 1)

    test_y = np.asarray(test_y, dtype=np.float32)
    test_x = np.asarray(test_x, dtype=np.float32)

    results = model.evaluate(test_x, test_y, verbose=1, return_dict=True)

    return results

In [40]:
results = eval(model)

100%|██████████| 828/828 [00:00<00:00, 339646.33it/s]

Fitting Tokenizer to dataset.



1475it [00:00, 14743.86it/s]

removing to from vocab
removing and from vocab
removing of from vocab
removing a from vocab
removing that from vocab
removing in from vocab
removing is from vocab
removing i from vocab
removing we from vocab
removing for from vocab
removing you from vocab
removing it from vocab
removing have from vocab
removing this from vocab
removing on from vocab
removing are from vocab
removing be from vocab
removing with from vocab
removing not from vocab
removing as from vocab
removing but from vocab
removing was from vocab
removing they from vocab
removing our from vocab
removing what from vocab
removing he from vocab
removing about from vocab
removing at from vocab
removing do from vocab
removing so from vocab
removing people from vocab
removing will from vocab
removing by from vocab
removing going from vocab
removing from from vocab
removing has from vocab
removing who from vocab
removing all from vocab
removing think from vocab
removing or from vocab
removing if from vocab
removing president 

400000it [00:27, 14289.00it/s]
100%|██████████| 6699/6699 [00:00<00:00, 458692.09it/s]


word2vec embeds have len 300
Translating vocabulary
Could not find 226 words in GLOVE.


100%|██████████| 167/167 [00:00<00:00, 4629.11it/s]


Converting text from test set to sequences.
6/6 [==============================] - 18s 3s/step - loss: 0.6924 - binary_accuracy: 0.7365 - true_positives: 74.0000 - true_negatives: 49.0000 - false_positives: 26.0000 - false_negatives: 18.0000


In [41]:
results

{'binary_accuracy': 0.7365269660949707,
 'false_negatives': 18.0,
 'false_positives': 26.0,
 'loss': 0.692355215549469,
 'true_negatives': 49.0,
 'true_positives': 74.0}

In [42]:
with open('/content/results.json', 'w') as results_file:
  json.dump(results, results_file, indent=4)